In [5]:
import pandas as pd
import numpy as np

In [8]:
dataset = pd.read_csv('./Suicide_Detection.csv')

In [9]:
dataset.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,3,Am I weird I don't get affected by compliments...,non-suicide
2,4,Finally 2020 is almost over... So I can never ...,non-suicide
3,8,i need helpjust help me im crying so hard,suicide
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",suicide


In [10]:
dataset['class'].replace({'suicide' : 1 , 'non-suicide' : 0}, inplace=True)

In [11]:
dataset.head()

,Unnamed: 0,text,class
0,2,Ex Wife Threatening SuicideRecently I left my ...,1
1,3,Am I weird I don't get affected by compliments...,0
2,4,Finally 2020 is almost over... So I can never ...,0
3,8,i need helpjust help me im crying so hard,1
4,9,"I’m so lostHello, my name is Adam (16) and I’v...",1


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset["text"], dataset["class"],  test_size=0.2)

In [29]:
import torch
print(torch.__version__)

2.0.1


In [32]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available, using", torch.cuda.get_device_name(device))
else:
    print("No GPU detected, using CPU for computations.")

No GPU detected, using CPU for computations.


In [18]:
all_strs = dataset["text"].values.tolist()
print(len(all_strs))
print(all_strs[0])

232074
Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it doesn't happen.


In [20]:
import spacy

In [ ]:
def get_dictionary(texts):
    dict_set = set()
    for text in texts:
        for word in text:
            dict_set.add(word)
    return dict_set;

In [ ]:
def text_pre_processing(texts):
    res = []
    nlp = spacy.load("en_core_web_sm")

    for text in texts:
        doc = nlp(text)
        stop = " ".join([token.text for token in doc if not token.is_stop])
        doc = nlp(stop)
        lem = " ".join([token.lemma_ for token in doc])
        ret.append(lem)
    
    return res

In [12]:
all_strs = dataset["text"].values.tolist()
dict_set = set()
for tweet in all_strs:
    for word in tweet:
        dict_set.add(word)

In [14]:
dict_set.add("<unk>")
len(dict_set)

5597

In [16]:
word_to_int = dict((word, i) for i, word in enumerate(dict_set))
int_to_word = dict((i, word) for i, word in enumerate(dict_set))

In [ ]:
# def construct_sequence(sequence_len, len):
#     x = []
#     y = []
#     for i in range (0, len - sequence_len):
#         x_ = full_text[i : i+sequence_len]
#         y_ = full_text[i+1 : i+sequence_len+1]

#         x.append([char_to_int[c] for c in x_])
#         y.append([char_to_int[c] for c in y_])
#     return x, y

In [ ]:
def one_hot(x, sequence_len, size):
    onehot = np.zeros((size, sequence_len, len(unique_chars)))
    for i in range(size):
        for j, index in enumerate(x[i]):
            onehot[i, j, index] = 1       
    return onehot

In [ ]:
X = dataset["text"]
Y = dataset["class"]

In [15]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(1, batch_size, self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

In [ ]:
def train(model, optimizer, criterion, train_loader, test_loader, num_epochs):
    